# **Import libraries**

In [2]:
pip install keras_tuner

     |████████████████████████████████| 96 kB 4.0 MB/s 
  Created wheel for kt-legacy: filename=kt_legacy-1.0.3-py3-none-any.whl size=9568 sha256=02c9b425cd2ac04a18510e026a8da6ce0e767b9367a36d203c4eceea7ccdf73c
  Stored in directory: /root/.cache/pip/wheels/38/5c/e0/13003e68c17f403af40b92a24d20171b95fef13b0fdaba833c
Successfully built kt-legacy


In [3]:
from tensorflow import keras

from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import os

import numpy as np
from numpy import concatenate

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from keras import optimizers
from tensorflow.keras.optimizers import Adam, SGD

from torch.optim import Adam, SGD
from sklearn.model_selection import train_test_split
from sklearn.semi_supervised import LabelPropagation

from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix, roc_auc_score, average_precision_score
import time
import itertools

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow import keras
from tensorflow.python.keras.datasets import cifar100

import keras_tuner as kt
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19


%matplotlib inline

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Loading data**

In [ ]:
! unzip /content/drive/MyDrive/CIFAR100.zip -d cifar100

In [ ]:
def unpickle(file):
    
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
        
    return dict

**Create data table**

In [ ]:
path_train = '/content/cifar100/train'
train_dict = unpickle(path_train)

path = '/content/cifar100/meta'
names_dict = unpickle(path)

fine_labels_list = train_dict[b'fine_labels']
coarse_labels_list = train_dict[b'coarse_labels']
data_list = train_dict[b'data']

fine_label_names_list = names_dict[b'fine_label_names']
coarse_label_names_list = names_dict[b'coarse_label_names']


In [ ]:
# Create the df_train dataframe
df_train = pd.DataFrame(fine_labels_list, columns=['fine_labels'])

# Create new columns
df_train['coarse_labels'] = coarse_labels_list
df_train['image_num'] = df_train.index

# Create the image_id column
def create_imageid(row):
    image_id = str(row['image_num']) +'.jpg'
    return image_id

df_train['image_id'] = df_train.apply(create_imageid, axis=1)

# Create the fine and coarse label names columns
def create_finelabelname(x):
    # this returns bytes: b'apple'
    name = fine_label_names_list[x]
    # convert bytes to string: 'apple'
    name = name.decode("utf-8") 
    return name

def create_coarselabelname(x):
    # this returns bytes: b'apple'
    name = coarse_label_names_list[x]
    # convert bytes to string: 'apple'
    name = name.decode("utf-8") 
    return name

df_train['fine_label_names'] = df_train['fine_labels'].apply(create_finelabelname)
df_train['coarse_label_names'] = df_train['coarse_labels'].apply(create_coarselabelname)
df_train['data'] = data_list.tolist()

# Remove unnecessary columns
df_train = df_train.drop('image_num', axis=1)

# Reorder the columns
cols = ['image_id', 'fine_label_names', 'fine_labels', 'coarse_label_names', 'coarse_labels', 'data']
df_train = df_train[cols]

df_train

In [ ]:
def change_to_array(X_train, X_test):
    new_data_train = []
    new_data_test = []
    X_data1 = np.array(X_train)
    X_data3 = np.array(X_test)
    for val in X_data1:
        new_data_train.append(val)
    for val in X_data3:
        new_data_test.append(val)
    return  np.array(new_data_train),np.array(new_data_test)

In [ ]:
def split_data(df,num_classes):
  X=df['data']
  y=df['fine_label_names']

  x_train, x_test, y_train, y_test=train_test_split(X,y,test_size=0.2)
  x_train, x_test=change_to_array(x_train, x_test)

  x_train = x_train.reshape(len(x_train),3,32,32).transpose(0,2,3,1)
  x_test = x_test.reshape(len(x_test),3,32,32).transpose(0,2,3,1)

  # Normalize the data. Before we need to connvert data type to float for computation.
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  x_train /= 255
  x_test /= 255

  # Convert class vectors to binary class matrices. This is called one hot encoding.
  y_train = keras.utils.to_categorical(y_train, num_classes)
  y_test = keras.utils.to_categorical(y_test, num_classes)


  return x_train, x_test, y_train, y_test

#MODEL I
**Defining the model architecture**

**parameters**

In [ ]:
n_folds=10
epochs=7
batch_size=32  #The default batch size of keras.
num_classes=10
#x_train.shape[1:]
input_shape=(32, 32, 3)
model_II=False

In [ ]:
def build_model(hp):

  #define the convnet
  model = Sequential()   

  model.add(Conv2D(
                  filters=16,
                  kernel_size=3,
                  padding='same',
                  activation='relu',
                  input_shape=input_shape))
  model.add(Conv2D(
                  filters=16,
                  padding='same',
                  activation='relu',
                  kernel_size=3))
  model.add(MaxPooling2D(pool_size=2))
  model.add(Dropout(rate=0.025))
  model.add(Conv2D(
                  filters=32,
                  kernel_size=3,
                  padding='same',
                  activation='relu'))
  model.add(Conv2D(
                  filters=64,
                  kernel_size=3,
                  padding='same',
                  activation='relu'))
  model.add(MaxPooling2D(pool_size=2,padding='same'))
  model.add(Dropout(rate=0.025))
  model.add(Flatten())
  model.add(Dense(units=128, activation='relu'))
  model.add(Dropout(rate=0.025))
  model.add(Dense(num_classes, activation='softmax'))
  if model_II==False:
    model.compile(
        optimizer=keras.optimizers.SGD(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
  else:
      model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy'])


  return model



 **Model training**

In [ ]:
def hyperparameters(x_train, y_train):
  tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    seed=1,
    max_trials=50,
    executions_per_trial=2,
    directory='search',
    project_name='cifar100') 

  tuner.search(x_train, y_train,
             epochs=3,
             validation_split=0.1) 

  # Get the optimal hyperparameters
  best_model=tuner.get_best_hyperparameters(num_trials=1)[0]
  model = tuner.hypermodel.build(best_model)

  return model,best_model.get('learning_rate')


In [ ]:
def calculateMeasures(model, x_test, y_test):
  measures = []

  start_time = time.time()
  y_pred=model.predict(x_test)
  measures.append(time.time()-start_time)

  pred=np.argmax(y_pred,axis=1)
  ground = np.argmax(y_test,axis=1)
  cnf_matrix = confusion_matrix(ground, pred)
  
  FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
  FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
  TP = np.diag(cnf_matrix)
  TN = cnf_matrix.sum() - (FP + FN + TP)

  FP = FP.astype(float)
  FN = FN.astype(float)
  TP = TP.astype(float)
  TN = TN.astype(float)

  #accuracy
  measures.append(accuracy_score(ground, pred))
  print("accuracy: ", measures[1])

  # Sensitivity, hit rate, recall, or true positive rate
  measures.append(recall_score(ground, pred, average="macro"))
  print("TPR-recall: ",measures[2])

  # Fall out or false positive rate
  FPR = FP/(FP+TN)
  FPR = np.average(FPR)
  measures.append(FPR)
  print("FPR: ",measures[3])

  #precision
  measures.append(precision_score(ground, pred, average="macro"))
  print("precision: ", measures[4])

  #AUC – Area Under the ROC Curve
  measures.append(roc_auc_score(y_test, model.predict_proba(x_test), multi_class='ovr'))
  print("AUC: ", measures[5])

  measures.append(average_precision_score(y_test, y_pred, average="micro"))
  print("PR-Curve: ", measures[6])
  return measures


In [ ]:
def run(model,x_train, y_train,x_test,y_test):
    fit_model_log=model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
    test_score = model.evaluate(np.array([x for x in x_test]), y_test)

    print(model.metrics_names[0] + ":"+ str(test_score[0]))
    print(model.metrics_names[1] + ":"+ str(test_score[1]))
    measures = calculateMeasures(model, x_test, y_test)
    return fit_model_log, measures

**Train the model with K-fold Cross Val**


In [ ]:
def run_model(datasetName, df, num_classes, results):
  model_history = [] 
  for i in range(n_folds):
    print("Training on Fold: ",i+1)
    # split into train and test
    x_train, x_test, y_train, y_test=split_data(df,num_classes)

    if model_II:
      x_train, x_test = normalize(x_train, x_test)

    # split train into labeled and unlabeled
    x_train_lab, x_test_unlab, y_train_lab, y_test_unlab = train_test_split(x_train, y_train, test_size=0.50, random_state=1, stratify=y_train)
    # create the training dataset input
    nsamples, nx, ny,nz = x_train_lab.shape
    x_train_lab = x_train_lab.reshape((nsamples,nx*ny*nz))

    nsamples, nx, ny,nz = x_test_unlab.shape
    x_test_unlab = x_test_unlab.reshape((nsamples,nx*ny*nz))

    x_train_mixed = concatenate((x_train_lab, x_test_unlab))

    # create "no label" for unlabeled data
    nolabel=[-1 for _ in range(len(y_test_unlab))]
    y_train_lab=np.argmax(y_train_lab, axis=-1)
    # recombine training dataset labels
    y_train_mixed = concatenate((y_train_lab, nolabel))

    start_time = time.time()
    # define model
    model = LabelPropagation()
    # fit model on training dataset
    model.fit(x_train_mixed, y_train_mixed)
    # get labels for entire training dataset data
    tran_labels = model.transduction_

    tran_labels=keras.utils.to_categorical(tran_labels, num_classes)
    x_train_mixed = x_train_mixed.reshape(len(x_train_mixed),3,32,32).transpose(0,2,3,1)


    # define supervised learning model
    best_model_hp,lr=hyperparameters(x_train_mixed, tran_labels)
    print(lr)
    fit_model_log, measures = run(best_model_hp,x_train_mixed, tran_labels,x_test,y_test)
    end_time = time.time() - start_time
    model_history.append(fit_model_log)
    print("======="*12, end="\n\n\n")
    results.loc[len(results)] = np.array([datasetName, 'label propagate', i+1, lr, measures[1], measures[2], measures[3], measures[4], measures[5], measures[6], end_time , measures[0]])

  return results

**run model on 20 dataset**

In [ ]:
df_train['coarse_label_names'].value_counts()
labels=df_train['coarse_label_names'].unique()
columnsName=['Dataset Name', 'Algoritem Name', 'Cross Validation', 'Hyper-Parameters Values', 'Accuracy', 'TPR', 'FPR', 'Precision', 'AUC', 'PR-Curve', 'Training Time', 'Inference Time']
results = pd.DataFrame(columns=columnsName)
datasetsName = []
# create excel writer object
writer = pd.ExcelWriter('output_model1.xlsx')


#df_x
for x in range(len(labels)):
  if x < 19:
    globals()["df_"+str(x)] = df_train.drop(df_train[(df_train.coarse_label_names != labels[x]) & (df_train.coarse_label_names != labels[x+1])].index)
    datasetName = 'Cifar100-'+labels[x]+'-'+labels[x+1]
  else:
    globals()["df_"+str(x)] = df_train.drop(df_train[(df_train.coarse_label_names != labels[x]) & (df_train.coarse_label_names != labels[0])].index)
    datasetName = 'Cifar100-'+labels[x]+'-'+labels[0]
  datasetsName.append(datasetName)
  globals()["df_"+str(x)].fine_label_names =pd.factorize(globals()["df_"+str(x)].fine_label_names)[0]
  results = run_model(datasetName, globals()["df_"+str(x)],num_classes, results)
  # write dataframe to excel
  results.to_excel(writer)
  # save the excel
  writer.save()

results

# **Model II**

**Improvement to the algorithm**

In [ ]:
def normalize(X_train,X_test):
  #this function normalize inputs for zero mean and unit variance
  # it is used when training a model.
  # Input: training set and test set
  # Output: normalized training set and test set according to the trianing set statistics.
  mean = np.mean(X_train,axis=(0,1,2,3))
  std = np.std(X_train, axis=(0, 1, 2, 3))
  # print(mean)
  # print(std)
  X_train = (X_train-mean)/(std+1e-7)
  X_test = (X_test-mean)/(std+1e-7)
  return X_train, X_test

In [ ]:
model_II=True
df_train['coarse_label_names'].value_counts()
labels=df_train['coarse_label_names'].unique()
columnsName=['Dataset Name', 'Algoritem Name', 'Cross Validation', 'Hyper-Parameters Values', 'Accuracy', 'TPR', 'FPR', 'Precision', 'AUC', 'PR-Curve', 'Training Time', 'Inference Time']
results = pd.DataFrame(columns=columnsName)
datasetsName = []
# create excel writer object
writer = pd.ExcelWriter('output_model2.xlsx')

for x in range(len(labels)):
  print(x)
  if x < 19:
    globals()["df_"+str(x)] = df_train.drop(df_train[(df_train.coarse_label_names != labels[x]) & (df_train.coarse_label_names != labels[x+1])].index)
    datasetName = 'Cifar100-'+labels[x]+'-'+labels[x+1]
  else:
    globals()["df_"+str(x)] = df_train.drop(df_train[(df_train.coarse_label_names != labels[x]) & (df_train.coarse_label_names != labels[0])].index)
    datasetName = 'Cifar100-'+labels[x]+'-'+labels[0]
  datasetsName.append(datasetName)
  globals()["df_"+str(x)].fine_label_names =pd.factorize(globals()["df_"+str(x)].fine_label_names)[0]
  results = run_model(datasetName, globals()["df_"+str(x)],num_classes, results)
  # write dataframe to excel
  results.to_excel(writer)
  # save the excel
  writer.save()
results





#**Model III**

**base model used is VGG19**- The pretrained weights from the imagenet challenge are used

In [ ]:
base_model = VGG19(include_top=False,weights='imagenet',input_shape=(32,32,3),classes=5)
#Lets add the final layers to these base models where the actual classification is done in the dense layers

def build_model_3(hp):
  model_3= Sequential()
  model_3.add(base_model) #Adds the base model (in this case vgg19 to model_3)
  model_3.add(Flatten()) #Since the output before the flatten layer is a matrix we have to use this function to get a vector of the form nX1 to feed it into the fully connected layers
  model_3.add(Dense(5,activation=('softmax'))) #This is the classification layer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  model_3.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
  return model_3




In [ ]:
def hyperparameters_3(x_train, y_train):
  tuner = kt.RandomSearch(
    build_model_3,
    objective='val_accuracy',
    seed=1,
    max_trials=50,
    executions_per_trial=2,
    directory='random_search',
    project_name='cifar100') 

  tuner.search(x_train, y_train,
             epochs=3,
             validation_split=0.1) 

  # Get the optimal hyperparameters
  best_model=tuner.get_best_hyperparameters(num_trials=1)[0]
  model = tuner.hypermodel.build(best_model)

  return model,best_model.get('learning_rate')

In [ ]:
def run_3(model,x_train, y_train,x_test,y_test):
    fit_model_log=model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
    test_score = model.evaluate(np.array([x for x in x_test]), y_test)
    print(model.metrics_names[0] + ":"+ str(test_score[0]))
    print(model.metrics_names[1] + ":"+ str(test_score[1]))
    measures = calculateMeasures(model, x_test, y_test)
    return fit_model_log

In [ ]:
def run_model_3(datasetName,df,num_classes,results):
  model_history = [] 
  for i in range(n_folds):
    print("Training on Fold: ",i+1)
    # split into train and test
    x_train, x_test, y_train, y_test=split_data(df,num_classes)
    # split train into labeled and unlabeled
    x_train_lab, x_test_unlab, y_train_lab, y_test_unlab = train_test_split(x_train, y_train, test_size=0.50, random_state=1, stratify=y_train)
    # create the training dataset input
    nsamples, nx, ny,nz = x_train_lab.shape
    x_train_lab = x_train_lab.reshape((nsamples,nx*ny*nz))

    nsamples, nx, ny,nz = x_test_unlab.shape
    x_test_unlab = x_test_unlab.reshape((nsamples,nx*ny*nz))

    x_train_mixed = concatenate((x_train_lab, x_test_unlab))

    # create "no label" for unlabeled data
    nolabel=[-1 for _ in range(len(y_test_unlab))]
    y_train_lab=np.argmax(y_train_lab, axis=-1)
    # recombine training dataset labels
    y_train_mixed = concatenate((y_train_lab, nolabel))
    start_time=time.time()
    # define model
    model = LabelPropagation()
    # fit model on training dataset
    model.fit(x_train_mixed, y_train_mixed)
    # get labels for entire training dataset data
    tran_labels = model.transduction_

    tran_labels=keras.utils.to_categorical(tran_labels, num_classes)
    x_train_mixed = x_train_mixed.reshape(len(x_train_mixed),3,32,32).transpose(0,2,3,1)


    # define supervised learning model
    best_model_hp,lr=hyperparameters_3(x_train_mixed, tran_labels)
    print(lr)
    fit_model_log, measures = run_3(best_model_hp,x_train_mixed, tran_labels,x_test,y_test)
    end_time=time.time()-start_time
    model_history.append(fit_model_log)
    print("======="*12, end="\n\n\n")
    results.loc[len(results)] = np.array([datasetName, 'CNN- ResNet50', i+1, lr, measures[1], measures[2], measures[3], measures[4], measures[5], measures[6], end_time , measures[0]])

  return results

In [ ]:
df_train['coarse_label_names'].value_counts()
labels=df_train['coarse_label_names'].unique()
columnsName=['Dataset Name', 'Algoritem Name', 'Cross Validation', 'Hyper-Parameters Values', 'Accuracy', 'TPR', 'FPR', 'Precision', 'AUC', 'PR-Curve', 'Training Time', 'Inference Time']
results = pd.DataFrame(columns=columnsName)
datasetsName = []

#df_x
for x in range(len(labels)):
  print(x)
  if x < 19:
    globals()["df_"+str(x)] = df_train.drop(df_train[(df_train.coarse_label_names != labels[x]) & (df_train.coarse_label_names != labels[x+1])].index)
    datasetName = 'Cifar100-'+labels[x]+'-'+labels[x+1]
  else:
    globals()["df_"+str(x)] = df_train.drop(df_train[(df_train.coarse_label_names != labels[x]) & (df_train.coarse_label_names != labels[0])].index)
    datasetName = 'Cifar100-'+labels[x]+'-'+labels[0]
  datasetsName.append(datasetName)
  globals()["df_"+str(x)].fine_label_names =pd.factorize(globals()["df_"+str(x)].fine_label_names)[0]
  results = run_model_3(datasetName, globals()["df_"+str(x)],num_classes, results)

results
  
# create excel writer object
writer = pd.ExcelWriter('output_model3.xlsx')
# write dataframe to excel
results.to_excel(writer)
# save the excel
writer.save()

# **Statistical significance testing of the results**

**Friedman test**

In [ ]:
from scipy.stats import friedmanchisquare

# Loading the example DataFrame.
performances = pd.read_csv('/content/drive/MyDrive/AUCResults.csv')
# First, we extract the algorithms names.
algorithms_names = performances.drop('Dataset Name', axis=1).columns
# Then, we extract the performances as a numpy.ndarray.
performances_array = performances[algorithms_names].values
# Finally, we apply the Friedman test.
friedmanchisquare(*performances_array)